In [1]:
#pip install polytope


In [2]:
#pip install pypoman

In [3]:
# pip install polytope
# pip install pypoman

import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull

Let's construct a reflexive 2D polytope (taken from the book "The Calabi–Yau Landscape", page 44)

In [4]:
V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])


Pypoman provides a form $0\le -Ax+b$

In [5]:
A, b = compute_polytope_halfspaces(V)

print(A)
print(b)

#print(compute_polytope_vertices(A, b))
print(convex_hull(V))

[[-0. -1.]
 [-1. -0.]
 [-1. -1.]
 [ 1.  1.]
 [ 1. -0.]]
[1. 1. 1. 1. 1.]
[array([-1,  2]), array([-1,  0]), array([ 0, -1]), array([ 1, -1]), array([1, 0])]


Polytope provides a form $Ax\le b$ which is equivalent to pypoman's form $0\le-Ax+b$

In [6]:
# P = qhull(V)

# print(P.A)
# print(P.b)
# #print(P.vertices)

Scipy yields an array $[normal, offset]$ that describes $Ax+b\le0$ forming the hyperplane equation of the facet see [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html)

In [7]:
hull = ConvexHull(V)
print(hull.equations)

[[-1.       0.      -1.     ]
 [ 0.70711  0.70711 -0.70711]
 [-0.70711 -0.70711 -0.70711]
 [ 1.       0.      -1.     ]
 [-0.      -1.      -1.     ]]


When using the negated form of pypoman's result, that is $Ax-b\le0$ then $b$ is the distance we are looking for.

The distances are defined in the hyperplane representation. The hyperplanes are given as $<u,v> \ge a$ where $u$ are points in the polytope lattice $M, v$ is a single point in the dual lattice $N$ and $a$ is the distance which is real number. If we just get the hyperplane representation then we automatically have the distances.

In [8]:
# A, b = compute_polytope_halfspaces(V)
# print(b)

Count integral points using polytope library.
An interesting reference is also [Effective lattice point counting in rational convex polytopes](https://www.sciencedirect.com/science/article/pii/S0747717104000422?via%3Dihub)

In [9]:
V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
polytope.ABS_TOL = 0
poly_qhull = qhull(V)
print(poly_qhull)
integral_points = enumerate_integral_points(poly_qhull)
integral_points = integral_points.transpose()
integral_points = integral_points.astype(int)
print(integral_points)

Single polytope 
  [[-1.      -0.     ] |    [[1.     ]
   [ 0.70711  0.70711] |     [0.70711]
   [-0.70711 -0.70711] x <=  [0.70711]
   [ 1.       0.     ] |     [1.     ]
   [ 0.      -1.     ]]|     [1.     ]]

[[ 0 -1]
 [ 1 -1]
 [-1  0]
 [ 0  0]
 [ 1  0]
 [-1  1]
 [ 0  1]
 [-1  2]]


The fitness function consists of two parts.

The first term is IP(Delta)-1, where IP(Delta) = 1 if Delta satisfies IP and 0 otherwise, adds a penalty if Delta doesn’t satisfy the IP property.

The second term sum(ai-1), where ai are the hyperplane distances of each face, adds a penalty if the distances of all the hyper planes aren’t at a distance 1 from the origin.

Note: Ensure that when you generate polytopes that you choose its points around the origin! Otherwise you have to translate the polytope such that its origin or one of its interior points becomes $(0,0,0,\ldots)$.

In [10]:
# #
# # 2D Example
# #
# # V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# V = np.array([[1, 3], [1, 1], [1, 2], [1, 1], [3, 1]])

# #
# # 3D Example
# #
# #good cube: its origin is (1,1,1)
# # V = np.array([[0,0,0], [2,0,0], [2,2,0], [0,2,0], [0,2,2], [2,2,2], [2,0,2], [0,0,2]])

# #good cube: its origin is (0,0,0)
# #
# # 5D Example
# #
# # V = np.array([[-1,0,1,1,-1], [0,1,-1,-1,1], [0,-3,-1,1,0], [-1,0,0,-1,0], [-1,1,-1,-1,1], [0,-1,1,1,0], [0,0,0,-1,0]])
# # V = np.array([[-2,0,1,0,-1], [1,0,1,-1,2], [-1,1,-1,-2,0], [1,-1,0,2,0], [1,0,0,-1,0], [0,0,0,2,-1]])

# A, b = compute_polytope_halfspaces(V)

# poly_qhull = qhull(V)
# print(poly_qhull)

# def grid_region(polyreg, res=None):
#     # grid corners
#     bbox = polyreg.bounding_box
#     # grid resolution
#     if res is None:
#         density = 8
#         res = [
#             math.ceil(density * (b - a))
#             for a, b in zip(*bbox)]
#     if len(res) != polyreg.dim:
#         raise ValueError((
#             "`len(res)` must equal the polytope's dimension "
#             "(which is {dim}), but instead `res` is:  {res}"
#             ).format(dim=polyreg.dim, res=res))
#     if any(n < 1 for n in res):
#         raise ValueError((
#             '`res` must contain `int` values >= 1, '
#             'instead `res` equals:  {res}'
#             ).format(res=res))
#     linspaces = list()
#     for a, b, n in zip(*bbox, res):
#         r = np.linspace(a, b, num=n)
#         linspaces.append(r)
#     points = np.meshgrid(*linspaces)
#     x = np.vstack(list(map(np.ravel, points)))
#     x = x[:, polyreg.contains(x, abs_tol=0)]
#     return (x, res)

# def enumerate_stricktly_integral_points(poly):
#     a, b = poly.bounding_box
#     a_int = np.floor(a)
#     b_int = np.ceil(b)
#     intervals = list(zip(a_int.flatten(), b_int.flatten()))
#     box = box2poly(intervals)
#     res = [int(b - a + 1) for a, b in intervals]
#     grid, _ = grid_region(box, res=res)
#     inside = poly.contains(grid, abs_tol=0)
#     return grid[:, inside]

# integral_points = enumerate_integral_points(poly_qhull)
# integral_points = integral_points.transpose()
# integral_points = integral_points.astype(int)

# stricktly_integral_points = enumerate_stricktly_integral_points(poly_qhull)
# stricktly_integral_points = stricktly_integral_points.transpose()
# stricktly_integral_points = stricktly_integral_points.astype(int)
# num_stricktly_integral_points = len(stricktly_integral_points)

# print("integral_points: {0}".format(integral_points.tolist()))
# print("stricktly_integral_points: {0}".format(stricktly_integral_points.tolist()))
# print("b: {0}".format(b))

# def fitness(ip_count, distances):
#     result = 0
#     if ip_count > 1:
#         result -= 1
#     for d in distances:
#         result -= abs(d-1)
#     return result

# print("fitness value: {0}".format(fitness(num_stricktly_integral_points, b)))

Some experimental stuff starts here:

# GA TEST

In [11]:
import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull
import random
from random import randint
import copy



class Population:
    def __init__(self, number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #self.generation_0 = Generation(chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        
        self.number_of_generations = number_of_generations
        self.all_generations = []
        #print("yyyyyyyyyyyyyyyyyyyyyyyyy")
        
        
        # try:
        #     self.all_generations.append(Generation(chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        # except:
        #     chrom_list = random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions)
        #     self.all_generations.append(Generation(chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        
        
        
        
        self.all_generations.append(Generation(chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        

        
        for curr_generation_number in range(0, number_of_generations-1):
            
            copy_prev_Generation_chrom_list = copy.deepcopy(self.all_generations[curr_generation_number-1].get_chrom_list())
            #copy_prev_Generation_chrom_list
            copy_prev_Generation_chrom_list = copy_prev_Generation_chrom_list[:int(len(copy_prev_Generation_chrom_list)/2)]
            parents = []
            #for curr_parent_choosing_round in range(0,len(self.all_generations[0])):
            #print(len(self.all_generations[0].get_chrom_list()))
            #print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
            while len(copy_prev_Generation_chrom_list) < len(self.all_generations[0].get_chrom_list()):

                parents = random.sample(copy_prev_Generation_chrom_list, 2)
                

                copy_prev_Generation_chrom_list.append(Chromosom.apply_crossover(parents[0], parents[1], crossover_type))
                
            #print(len(copy_prev_Generation_chrom_list))
                
            #if len(copy_generation) < len(self.all_generations[0]
            


            copy_prev_Generation_chrom_list.sort(key=lambda x: x.fitness, reverse=True)

            self.all_generations.append(Generation(copy_prev_Generation_chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        
    def get_all_generations(self):
        return self.all_generations


class Generation:
    num_points_per_polygon = 0
    num_chroms_per_Generation = 0
    num_dimensions = 0 # 3
    min_value_per_point = 0  # -3
    max_value_per_point = 0  # 3
    
    def __init__(self, chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        
        Generation.num_points_per_polygon = num_points_per_polygon
        Generation.num_chroms_per_Generation = num_chroms_per_Generation
        Generation.num_dimensions = num_dimensions # 3
        Generation.min_value_per_point = min_value_per_point  # -3
        Generation.max_value_per_point = max_value_per_point  # 3

        

        if chrom_list == None:
            self.chrom_list = self.gen_first_generation(num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        else:
            self.chrom_list = chrom_list

    def get_chrom_list(self):
        return self.chrom_list
    
    def get_dimension():
        return Generation.num_dimensions

    def get_min_value_per_point():
        return Generation.min_value_per_point
    
    def get_max_value_per_point():
        return Generation.max_value_per_point




    def gen_first_generation(self, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #num_points_per_polygon = 5
        #num_chroms_per_Generation = 10
        #num_dimensions = num_dimensions

        polygon_points = []
        generation_polygons = []

        for j in range(0, num_chroms_per_Generation):
            #chromosome_test_points=[]
            polygon_points = []
            for i in range(0,num_points_per_polygon):
                polygon_points.append(random.sample(range(min_value_per_point, max_value_per_point), num_dimensions))
            chromosome_polygon_points = np.array(polygon_points)
            #print(polygon_points)
            chromosom_test = Chromosom(chromosome_polygon_points)

            generation_polygons.append(chromosom_test)
            generation_polygons.sort(key=lambda x: x.fitness, reverse=True)

        return generation_polygons

class Chromosom:
    chrom_count = 0
    
    def __init__(self, points):
        

        
        result = None
        while result is None:
            try:
                internal_points = []
                #print("###############################################")
                Chromosom.chrom_count = Chromosom.chrom_count + 1
                #print(Chromosom.chrom_count)
                
                self.gen_list = points
                self.q_hull = qhull(self.gen_list)
                self.A, self.b_half_space = compute_polytope_halfspaces(self.gen_list)

                self.num_stricktly_integral_points = self.gen_strictly_integral_points(self.q_hull)

                self.fitness = self.calc_fitness(self.num_stricktly_integral_points, self.b_half_space)
                #print(fitness)
                result = 1
            except:
                points = random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions)
                pass

        
        
        
        
        
        
        internal_points = []
        #print("###############################################")
        Chromosom.chrom_count = Chromosom.chrom_count + 1
        #print(Chromosom.chrom_count)
        
        self.gen_list = points
        self.q_hull = qhull(self.gen_list)
        self.A, self.b_half_space = compute_polytope_halfspaces(self.gen_list)

        self.num_stricktly_integral_points = self.gen_strictly_integral_points(self.q_hull)

        self.fitness = self.calc_fitness(self.num_stricktly_integral_points, self.b_half_space)
        #print(fitness)

    # def get_q_hull(self):
    #     return self.q_hull

    # def get_b_half_space(self):
    #     return self. b_half_space
    
    def get_gen_list(self):
        return self.gen_list
    
    def get_fitness(self):
        return self.fitness

    def get_gen_list(self):
        return self.gen_list
    


    def grid_region(self, polyreg, res=None):
        # grid corners
        bbox = polyreg.bounding_box
        # grid resolution
        if res is None:
            density = 8
            res = [
                math.ceil(density * (b - a))
                for a, b in zip(*bbox)]
        if len(res) != polyreg.dim:
            raise ValueError((
                "`len(res)` must equal the polytope's dimension "
                "(which is {dim}), but instead `res` is:  {res}"
                ).format(dim=polyreg.dim, res=res))
        if any(n < 1 for n in res):
            raise ValueError((
                '`res` must contain `int` values >= 1, '
                'instead `res` equals:  {res}'
                ).format(res=res))
        linspaces = list()
        for a, b, n in zip(*bbox, res):
            r = np.linspace(a, b, num=n)
            linspaces.append(r)
        points = np.meshgrid(*linspaces)
        x = np.vstack(list(map(np.ravel, points)))
        x = x[:, polyreg.contains(x, abs_tol=0)]
        return (x, res)

    def enumerate_stricktly_integral_points(self, poly):
        a, b = poly.bounding_box
        a_int = np.floor(a)
        b_int = np.ceil(b)
        intervals = list(zip(a_int.flatten(), b_int.flatten()))
        box = box2poly(intervals)
        res = [int(b - a + 1) for a, b in intervals]
        grid, _ = self.grid_region(box, res=res)
        inside = poly.contains(grid, abs_tol=0)
        return grid[:, inside]

    def gen_strictly_integral_points(self, q_hull):
        integral_points = enumerate_integral_points(q_hull)
        integral_points = integral_points.transpose()
        integral_points = integral_points.astype(int)

        stricktly_integral_points = self.enumerate_stricktly_integral_points(q_hull)
        stricktly_integral_points = stricktly_integral_points.transpose()
        stricktly_integral_points = stricktly_integral_points.astype(int)
        num_stricktly_integral_points = len(stricktly_integral_points)
        return num_stricktly_integral_points

    def calc_fitness(self, ip_count, distances):
        result = 0
        if ip_count > 1:
            result -= 1
        for d in distances:
            result -= abs(d-1)
        
        if result > 0:
            result = result*(-1)

        #print(result)
        return result  
    
    def apply_crossover(chromosom_1, chromosom_2, crossover_type):
        
        if chromosom_1.get_fitness() > chromosom_2.get_fitness():
            chromosom_1_points = chromosom_1.get_gen_list()
            chromosom_2_points = chromosom_2.get_gen_list()
        else:
            chromosom_2_points = chromosom_1.get_gen_list()
            chromosom_1_points = chromosom_2.get_gen_list()
        
        
        
        crossover_type = crossover_type

        new_chromosom_gen_list = []
        if crossover_type == 1:
            
            if len(chromosom_1_points)>len(chromosom_2_points):
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
            else:
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

            #new_gen = []
            for i in range(0,length_new_chromosom):
                if i < random_crossover_point:
                    new_gen = []
                    for j in chromosom_1_points[i]:
                        new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
                else:
                    new_gen = []
                    for j in chromosom_2_points[i]:
                        new_gen.append(j)

                new_chromosom_gen_list.append(new_gen)

        try:
            new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
        except:
            result = None
            while result is None:
                try:
                    new_chromosom_gen_list = random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions)
                    new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
                    print("try")
                    result = 1
                except:
                    pass
                
        return new_chromosom


In [12]:

# test_generation = Generation(None, 5, 10, 3, -3, 3)

# test_polygons = test_generation.get_chrom_list()


# for i in test_polygons:
#     print(i.get_fitness())
#     print(i.get_gen_list())


    

In [13]:
# Population Test:

# test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polygon, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
test_population = Population(30, 0, 1, 0, None, 5, 10, 3, -3, 3)



In [14]:
test_population_generations = test_population.get_all_generations()
test_population_generations_chroms = test_population_generations[1].get_chrom_list()
print(test_population_generations_chroms)

all_fitness = []
for generation in test_population.get_all_generations():
    fitness = []
    for chrom in generation.get_chrom_list():
        fitness.append(chrom.get_fitness())
    
    all_fitness.append(fitness)
    print(fitness)

#print(all_fitness)

[<__main__.Chromosom object at 0x000001C404E225D0>, <__main__.Chromosom object at 0x000001C404E18E50>, <__main__.Chromosom object at 0x000001C404AB6F10>, <__main__.Chromosom object at 0x000001C3BC232890>, <__main__.Chromosom object at 0x000001C404E22E10>, <__main__.Chromosom object at 0x000001C3BC5DEAD0>, <__main__.Chromosom object at 0x000001C404E23910>, <__main__.Chromosom object at 0x000001C3B8BFA650>, <__main__.Chromosom object at 0x000001C404DC9810>, <__main__.Chromosom object at 0x000001C404E08990>]
[-4.0, -7.666666666666666, -15.666666666666707, -20.75, -21.75, -23.0, -23.0, -31.0, -33.0, -57.19999999999999]
[-2.0, -2.0, -4.0, -7.666666666666666, -8.600000000000001, -15.666666666666707, -18.200000000000014, -19.0, -20.75, -21.75]
[-2.000000000000001, -4.0, -7.666666666666666, -15.666666666666707, -17.333333333333332, -20.124999999999996, -20.75, -21.75, -25.333333333333336, -26.999999999999993]
[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -4.0, -7.666666666666666, -8.600000000000001, -8

In [15]:
# a = [1,2,3]
# b = a.copy()
# #b = []
# #b.append(a[0])
# #b.append(a[1])
# a[1] = 5

# print(a)
# print(b)



In [16]:
# Crossover test - to be checked

# # #def apply_crossover(chromosom_1, chromosom_2, crossover_type):
# chromosom_1_points = test_polygons[1].get_gen_list()
# chromosom_2_points = test_polygons[0].get_gen_list()
# # crossover_type = 1

# # new_chromosom_gen_list = []
# # if crossover_type == 1:
    

# #     if len(chromosom_1_points)>len(chromosom_2_points):
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
# #     else:
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

# #     #new_gen = []
# #     for i in range(0,length_new_chromosom):
# #         if i <= random_crossover_point:
# #             new_gen = []
# #             for j in chromosom_1_points[i]:
# #                 new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
# #         else:
# #             new_gen = []
# #             for j in chromosom_2_points[i]:
# #                 new_gen.append(j)

# #         new_chromosom_gen_list.append(new_gen)

# # new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
# #return new_chromosom

# new_chromosom = Chromosom.apply_crossover(test_polygons[1], test_polygons[0], 1)

# #print(random_crossover_point)
# print(chromosom_1_points)
# print(chromosom_2_points)
# print(new_chromosom.get_gen_list())
# print(new_chromosom.get_fitness())


In [17]:
# import numpy as np
# import polytope

# # halfspace representation computed using `polytope`
# # from the vertex representation given in the question
# vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# poly = polytope.qhull(vertices)
# # first halfspace representation
# A = np.array([
#     [0, -1],
#     [-1, 0],
#     [-1, -1],
#     [1, 1],
#     [1, 0]])
# b = np.array([1, 1, 1, 1, 1])
# question_poly_1 = polytope.Polytope(A, b)
# # second halfspace representation
# A = np.array([
#     [-0.70711, -0.70711],
#     [-1, -0],
#     [0.70711, 0.70711],
#     [0, -1],
#     [1, 0]])
# b = np.array([0.70711, 1, 0.70711, 1, 1])

# question_poly_2 = polytope.Polytope(A, b)
# # check that all the above halfspace representations
# # represent the same polytope
# assert poly == question_poly_1, (poly, question_poly_1)
# assert poly == question_poly_2, (poly, question_poly_2)

In [18]:
# inner_point = np.array([0,0]) # in this case it is the origin
# fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
# fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
# fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

# d1 = polytope.quickhull.distance(inner_point, fac1)
# d2 = polytope.quickhull.distance(inner_point, fac2)
# d3 = polytope.quickhull.distance(inner_point, fac3)
# print(d1)
# print(d2)
# print(d3)

Let's visualize our 2D polytope. For this purpose we use the [Polytope Library](https://github.com/tulip-control/polytope)

In [19]:
# project into 2D



In [20]:

# print(polytope.polytope.cheby_ball(P))

# min_V = np.min(V, axis=0) #min val of each cols
# min_x = min_V[0]
# min_y = min_V[1]
# max_V = np.max(V, axis=0) #max val of each cols
# max_x = max_V[0]
# max_y = max_V[1]

# ax = P.plot(linestyle="solid", linewidth=1, color="lightblue", edgecolor="red")
# ax.set_xlim([min_x-1, max_x+1])
# ax.set_ylim([min_y-1, max_y+1])
# plt.show()